In [2]:
from langchain.document_loaders import YoutubeLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.chains import LLMChain
from langchain.vectorstores import FAISS
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv("try.env"))
from langchain import PromptTemplate
from langchain.schema import StrOutputParser
from langchain.schema.runnable import RunnablePassthrough

In [3]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings
embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")

In [4]:
def vectordb_from_youtube(video_url:str) -> FAISS:
    loader = YoutubeLoader.from_youtube_url(video_url)
    transcript = loader.load()

    text_splitter = RecursiveCharacterTextSplitter(chunk_size = 800, chunk_overlap = 80)
    docs = text_splitter.split_documents(transcript)

    db = FAISS.from_documents(docs, embeddings)
    retriever = db.as_retriever()

    return retriever


In [5]:
video_url = "https://www.youtube.com/watch?v=CcfF3WRVhoM"
db = vectordb_from_youtube(video_url)
print(db)

tags=['FAISS', 'GoogleGenerativeAIEmbeddings'] vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x000002793F91ECA0> search_kwargs={}


In [6]:
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

In [7]:
from langchain.agents import load_tools
from langchain.agents import initialize_agent
from langchain.agents import AgentType


In [ ]:
def get_response (db, question,k = 4):
    docs = db.invoke(question, k = k)
    docs_page_content = " ".join([d.page_content for d in docs])
   
    llm = ChatGoogleGenerativeAI(model = "gemini-pro",temperature = 0.75, top_p = 0.85)
    prompt_temp = PromptTemplate(
        input_variable = ["question" , "docs"],
        template = """You are a helpful assistant who can answer questions
        based on video transcript and tools. You can answer every question.
        The tools you can use are wikipedia and llm_math.
        Answer the following question: {question}
        By using the tools or searching the following video transcript : {docs} 
        If the video transcript does not have informationaabout the question, use the tools to answer the question.
        Please answer to the point. You may not always need tools.
    
        The answers should not be more than 4 lines."""
    )

    tools = load_tools([ "wikipedia", "llm-math"], llm = llm)
    llm_prompt = prompt_temp
    llm_with_tools = llm.bind_tools(tools)
    rag_chain = (
        {"docs": db | format_docs, "question": RunnablePassthrough()}
        | llm_prompt
        | llm_with_tools
        | StrOutputParser()
    )
    
    return rag_chain.invoke(question)
    

In [9]:
get_response(db,"What was the reason of the hazard?")

The pilots failed to initiate a holding pattern to reduce altitude, which was part of the approach checklist.
